In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch import nn, optim
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt

# DataPath
- パスの管理を任せる
- KaggleやColaboratoryで実行する場合は修正が必要

In [ ]:
from enum import Enum
from pathlib import Path


class DataPath(Enum):
    Root = Path('../')
    # Input = Root / 'input'
    Input = Root / 'input/digit-recognizer'  # KaggleNotebookに対応
    Submission = Root / 'submissions'

    TrainCsv = Input / 'train.csv'
    TestCsv = Input / 'test.csv'

    SubmissionCsv = Input / 'sample_submission.csv'

In [ ]:
# ファイルの存在を確認する
assert DataPath.TrainCsv.value.exists()
assert DataPath.TestCsv.value.exists()

# Dataset

In [ ]:
import numpy as np
from torch.utils.data import Dataset


class TrainDataset(Dataset):
    def __init__(self, X: np.ndarray, y: np.ndarray):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        image = self.X[idx].reshape(1, 28, 28).astype(np.float32)

        label = self.y[idx]

        return image, label


class TestDataset(Dataset):
    def __init__(self, X: np.ndarray):
        self.X = X

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx].reshape(1, 28, 28).astype(np.float32)


In [ ]:
def prepare_data_loaders(batch_size):
    train = pd.read_csv(DataPath.TrainCsv.value)

    X = train.iloc[:, 1:].values
    y = train.iloc[:, 0].values
    X_train, X_valid, y_train, y_valid = train_test_split(X,
                                                          y,
                                                          train_size=0.8,
                                                          random_state=0)
    train_dataset = TrainDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    valid_dataset = TrainDataset(X_valid, y_valid)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, valid_loader

In [ ]:
torch.manual_seed(0)

BATCH_SIZE = 64

# 1. DatasetとDataLoader
train_loader, valid_loader = prepare_data_loaders(BATCH_SIZE)

# Device

In [ ]:
# GPUが利用できるなら使う
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(f'Use {device}')

# Model

In [ ]:
import torch
from torch import nn


class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 3, kernel_size=(2, 2))
        self.conv2 = nn.Conv2d(3, 6, kernel_size=(2, 2))

        self.fc1 = nn.Linear(6 * 6 * 6, 100)
        self.fc2 = nn.Linear(100, 50)
        self.fc3 = nn.Linear(50, 10)

        self.relu = nn.ReLU(inplace=True)
        self.max_pool2d = nn.MaxPool2d(kernel_size=(2, 2))

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.max_pool2d(x)

        x = self.conv2(x)
        x = self.relu(x)
        x = self.max_pool2d(x)

        x = x.view(-1, 6 * 6 * 6)

        x = self.fc1(x)
        x = self.relu(x)

        x = self.fc2(x)
        x = self.relu(x)
        
        x = self.fc3(x)

        return x


In [ ]:
# 2. モデル(ネットワーク)
model: nn.Module = SimpleCNN()
model.to(device)  # GPUに転送

# 最適化アルゴリズムと損失関数
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

# 学習


In [ ]:
def run_train_epoch(model, train_loader, criterion, optimizer, epoch):
    model.train()

    running_loss = 0.0
    running_acc = 0.0

    for images, labels in train_loader:
        # GPUに転送
        images, labels = images.to(device), labels.to(device)

        # 勾配初期化
        optimizer.zero_grad()

        # 順伝播計算
        outputs = model(images)

        # 損失の計算
        loss = criterion(outputs, labels)

        # Backward
        loss.backward()

        # 重みの更新
        optimizer.step()
        
        _, labels_pred = torch.max(outputs, dim=1)
        running_loss += loss.item() * images.size(0)
        running_acc += accuracy_score(labels.cpu().clone().detach().numpy(), labels_pred.cpu().clone().detach().numpy()) * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = running_acc / len(train_loader.dataset)
    loss_train_list.append(epoch_loss)
    acc_train_list.append(epoch_acc)
    print(f'Epoch: {epoch} Train Loss: {epoch_loss:.4f} Train Acc: {epoch_acc:.4f}')


def run_valid_epoch(model, valid_loader, criterion, epoch):
    model.eval()

    running_loss = 0.0
    running_acc = 0.0

    with torch.no_grad():
        for images, labels in valid_loader:
            # GPUに転送
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)

            loss = criterion(outputs, labels)
            
            _, labels_pred = torch.max(outputs, dim=1)
            running_loss += loss.item() * images.size(0)
            running_acc += accuracy_score(labels.cpu().clone().detach().numpy(), labels_pred.cpu().clone().detach().numpy()) * images.size(0)

    epoch_loss = running_loss / len(valid_loader.dataset)
    epoch_acc = running_acc / len(valid_loader.dataset)
    loss_valid_list.append(epoch_loss)
    acc_valid_list.append(epoch_acc)
    
    print(f'Epoch: {epoch} Valid Loss: {epoch_loss:.4f} Valid Acc: {epoch_acc:.4f}')

In [ ]:
# 3. 学習
#torch.manual_seed(0)
NUM_EPOCHS = 5

# 評価の記録用リスト
loss_train_list = []
acc_train_list = []
loss_valid_list = []
acc_valid_list = []

for epoch in tqdm(range(1, NUM_EPOCHS + 1)):
    run_train_epoch(model, train_loader, criterion, optimizer, epoch)
    run_valid_epoch(model, valid_loader, criterion, epoch)

In [ ]:
plt.plot(range(1, len(loss_train_list) + 1), loss_train_list, label='loss_train')
plt.plot(range(1, len(loss_valid_list) + 1), loss_valid_list, label='loss_valid')
plt.xlabel('epoch')
plt.ylabel('CEL')
plt.legend()
plt.show()

In [ ]:
plt.plot(range(1, len(acc_train_list) + 1), acc_train_list, label='acc_train')
plt.plot(range(1, len(acc_valid_list) + 1), acc_valid_list, label='acc_valid')
plt.xlabel('epoch')
plt.ylabel('Accuracy')
plt.legend()

# テストデータでの予測とSubmission

In [ ]:
def make_predictions(model, test_loader):
    model.eval()
    predictions = np.array([], dtype=np.int)

    with torch.no_grad():
        for images in test_loader:
            # GPUに転送
            images = images.to(device)

            outputs = model(images)

            _, y_pred = torch.max(outputs, dim=1)
            y_pred_label = y_pred.cpu().numpy()

            predictions = np.append(predictions, y_pred_label)

    return predictions

In [ ]:
# 4. TestDataでの予測
df_test = pd.read_csv(DataPath.TestCsv.value)
X_test = df_test.values

test_dataset = TestDataset(X_test)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

predictions = make_predictions(model, test_loader)

In [ ]:
import datetime


def make_submission_file(model, predictions):
    submit_data = pd.read_csv(DataPath.SubmissionCsv.value)
    submit_data['Label'] = predictions

    yymmddhhmmss = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    model_name = model.__class__.__name__

    # ex: '20201231_174530_SimpleNet.csv
    save_submission_path = f'{yymmddhhmmss}_{model_name}.csv'

    submit_data.to_csv(save_submission_path, index=False)
    print(f'Saved {save_submission_path}')


In [ ]:
# submissionの作成
make_submission_file(model, predictions)

おわり